# Preparación dato Amazon Reviews

In [ ]:
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8


In [ ]:
# Load data
path = "drive/MyDrive/TFG_1/aws_data/dataset_zh_test.json"
df = pd.read_json(path, lines = True).rename(columns={'stars':'star_rating'})
df = df.sample(frac=1).reset_index(drop=True)

## Creamos columnas con combinaciones para futuras clasificaciones

df['text_title'] = df['review_title'] + " " + df['review_body']
df['text_title_cat'] = df['review_title'] + " " + df['review_body'] + " " + df['product_category']
df['text_cat'] = df['review_body'] + " " + df['product_category']
df.head()

,review_id,product_id,reviewer_id,star_rating,review_body,review_title,language,product_category,text_title,text_title_cat,text_cat
0,zh_0001114,product_zh_0892899,reviewer_zh_0904382,3,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？,买错了，能贴前面吗？,zh,wireless,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？ wireless
1,zh_0215277,product_zh_0978385,reviewer_zh_0886185,3,这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般,zh,digital_ebook_purchase,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_pu...,这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_purchase
2,zh_0095718,product_zh_0959813,reviewer_zh_0947587,1,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...,亚马逊运输打包究竟是按什么标准来的,zh,toy,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...
3,zh_0164458,product_zh_0516966,reviewer_zh_0096903,3,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...,功能确实强大，细节方面不是太好,zh,other,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...
4,zh_0513285,product_zh_0963328,reviewer_zh_0663012,3,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...,有本事你把口袋再做大点试试！,zh,grocery,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...


In [ ]:
df.shape

(5000, 11)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         5000 non-null   object
 1   product_id        5000 non-null   object
 2   reviewer_id       5000 non-null   object
 3   star_rating       5000 non-null   int64 
 4   review_body       5000 non-null   object
 5   review_title      5000 non-null   object
 6   language          5000 non-null   object
 7   product_category  5000 non-null   object
 8   text_title        5000 non-null   object
 9   text_title_cat    5000 non-null   object
 10  text_cat          5000 non-null   object
dtypes: int64(1), object(10)
memory usage: 429.8+ KB


In [ ]:
import itertools
class LabelEncoder(object):
    """Label encoder for tag labels."""
    def __init__(self, class_to_index={}):
        self.class_to_index = class_to_index
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())

    def __len__(self):
        return len(self.class_to_index)

    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"

    def fit(self, y):
        classes = np.unique(df['product_category'])
        for i, class_ in enumerate(classes):
            self.class_to_index[class_] = i
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        return self

    def encode(self, y):
        encoded = np.zeros((len(y)), dtype=int)
        for i, item in enumerate(y):
            encoded[i] = self.class_to_index[item]
        return encoded

    def decode(self, y):
        classes = []
        for i, item in enumerate(y):
            classes.append(self.index_to_class[item])
        return classes

    def save(self, fp):
        with open(fp, 'w') as fp:
            contents = {'class_to_index': self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, 'r') as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [ ]:
# Encode
label_encoder = LabelEncoder()
label_encoder.fit(df['product_category'])
NUM_CLASSES = len(label_encoder)
label_encoder.class_to_index

{'apparel': 0,
 'automotive': 1,
 'baby_product': 2,
 'beauty': 3,
 'book': 4,
 'camera': 5,
 'digital_ebook_purchase': 6,
 'drugstore': 7,
 'electronics': 8,
 'grocery': 9,
 'home': 10,
 'home_improvement': 11,
 'jewelry': 12,
 'kitchen': 13,
 'luggage': 14,
 'musical_instruments': 15,
 'office_product': 16,
 'other': 17,
 'pc': 18,
 'personal_care_appliances': 19,
 'pet_products': 20,
 'shoes': 21,
 'sports': 22,
 'toy': 23,
 'video_games': 24,
 'watch': 25,
 'wireless': 26}

In [ ]:
df['product_category_cod']= label_encoder.encode(df['product_category'])
df.head()

,review_id,product_id,reviewer_id,star_rating,review_body,review_title,language,product_category,text_title,text_title_cat,text_cat,product_category_cod
0,zh_0001114,product_zh_0892899,reviewer_zh_0904382,3,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？,买错了，能贴前面吗？,zh,wireless,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？ wireless,26
1,zh_0215277,product_zh_0978385,reviewer_zh_0886185,3,这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般,zh,digital_ebook_purchase,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_pu...,这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_purchase,6
2,zh_0095718,product_zh_0959813,reviewer_zh_0947587,1,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...,亚马逊运输打包究竟是按什么标准来的,zh,toy,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...,23
3,zh_0164458,product_zh_0516966,reviewer_zh_0096903,3,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...,功能确实强大，细节方面不是太好,zh,other,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...,17
4,zh_0513285,product_zh_0963328,reviewer_zh_0663012,3,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...,有本事你把口袋再做大点试试！,zh,grocery,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...,9


In [ ]:
## Change index of stars to avoid problems  [1-5] -- [0-4]
cats=sorted(df.star_rating.unique())
dic_cat=dict(zip(cats,range(len(cats))))

df.loc[:,'star_rating']=df.star_rating.apply(lambda x: dic_cat[x])

In [ ]:
print( sorted(df.product_category_cod.unique()))
print(sorted(df.star_rating.unique()))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[0, 1, 2, 3, 4]


In [ ]:
## cuando procesamos test para tambien conseguir validation 
#df_val, df_test = train_test_split(df, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
df.shape, 
#df_val.shape, df_test.shape

((5000, 12),)

In [ ]:
#df.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/train/amazonEN_train.csv', index = False, header=True)
#df_val.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonZH_valid.csv', index = False, header=True)
#df_test.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/test/amazonZH_test.csv', index = False, header=True)


# Preparación dato Amazon Reviews para clasificación simultánea de categoría y rating

In [ ]:
df.shape,# df_val.shape,# df_test.shape

((5000, 12),)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [ ]:
mlb.fit_transform([df['star_rating'], df['product_category_cod']])

array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1]])

In [ ]:
prueba = pd.DataFrame(mlb.fit_transform(df[['star_rating','product_category_cod']].values), columns=mlb.classes_, index=df.index)

In [ ]:
df['star_rating_str'] = df['star_rating'].astype(str)


In [ ]:
prueba = pd.DataFrame(mlb.fit_transform(df[['star_rating_str','product_category']].values), columns=mlb.classes_, index=df.index)

In [ ]:
prueba

,0,1,2,3,4,apparel,automotive,baby_product,beauty,book,camera,digital_ebook_purchase,drugstore,electronics,grocery,home,home_improvement,jewelry,kitchen,luggage,musical_instruments,office_product,other,pc,personal_care_appliances,pet_products,shoes,sports,toy,video_games,watch,wireless
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = pd.concat([df, prueba], axis=1,)

In [ ]:
df

,review_id,product_id,reviewer_id,star_rating,review_body,review_title,language,product_category,text_title,text_title_cat,text_cat,product_category_cod,star_rating_str,0,1,2,3,4,apparel,automotive,baby_product,beauty,book,camera,digital_ebook_purchase,drugstore,electronics,grocery,home,home_improvement,jewelry,kitchen,luggage,musical_instruments,office_product,other,pc,personal_care_appliances,pet_products,shoes,sports,toy,video_games,watch,wireless
0,zh_0001114,product_zh_0892899,reviewer_zh_0904382,2,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？,买错了，能贴前面吗？,zh,wireless,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？ 买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在...,买错了，能贴前面吗？刚刚发现是后膜，前面的膜呢？要后膜干嘛？这个能贴在前面吗？ wireless,26,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,zh_0215277,product_zh_0978385,reviewer_zh_0886185,2,这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般,zh,digital_ebook_purchase,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已,一般 这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_pu...,这本收入的短篇一般，不全是推理，像是记录某些现象而已 digital_ebook_purchase,6,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,zh_0095718,product_zh_0959813,reviewer_zh_0947587,0,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...,亚马逊运输打包究竟是按什么标准来的,zh,toy,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,亚马逊运输打包究竟是按什么标准来的 一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验...,一直信任亚马逊也经常在这上面买东西，但是这次真的购物体验极差，一个纸盒包的东西放这么一个袋子...,23,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,zh_0164458,product_zh_0516966,reviewer_zh_0096903,2,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...,功能确实强大，细节方面不是太好,zh,other,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,功能确实强大，细节方面不是太好 最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调...,最不喜欢的一点就是看视频的时候，右半边偶尔有明显的视频界面调整，幅度在一两毫米左右，很别扭；...,17,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,zh_0513285,product_zh_0963328,reviewer_zh_0663012,2,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...,有本事你把口袋再做大点试试！,zh,grocery,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,有本事你把口袋再做大点试试！ 味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么...,味道将就，就是老板你敢再把口袋做大点吗？那么大一个口袋就装那么几颗果子，你当你卖的是膨化食品...,9,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,zh_0190213,product_zh_0441221,reviewer_zh_0024230,2,有错字，而且错字很低级，都是很简单的字，但字数不多。有的语句不通。,此书有问题,zh,book,此书有问题 有错字，而且错字很低级，都是很简单的字，但字数不多。有的语句不通。,此书有问题 有错字，而且错字很低级，都是很简单的字，但字数不多。有的语句不通。 book,有错字，而且错字很低级，都是很简单的字，但字数不多。有的语句不通。 book,4,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,zh_0389580,product_zh_0330873,reviewer_zh_0411268,1,噪音大，上面还写个静音，简直无语，不写静音两字还没那么气人。,无语的噪音,zh,home,无语的噪音 噪音大，上面还写个静音，简直无语，不写静音两字还没那么气人。,无语的噪音 噪音大，上面还写个静音，简直无语，不写静音两字还没那么气人。 home,噪音大，上面还写个静音，简直无语，不写静音两字还没那么气人。 home,10,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,zh_0966073,product_zh_0520453,reviewer_zh_0021549,1,穿起来不是很好看，我不太懂关于衣服面料的问题，就是感觉怪怪的，不是很舒服。,不好评价,zh,apparel,不好评价 穿起来不是很好看，我不太懂关于衣服面料的问题，就是感觉怪怪的，不是很舒服。,不好评价 穿起来不是很好看，我不太懂关于衣服面料的问题，就是感觉怪怪的，不是很舒服。 app...,穿起来不是很好看，我不太懂关于衣服面料的问题，就是感觉怪怪的，不是很舒服。 apparel,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,zh_0536518,product_zh_0369005,reviewer_zh_0656989,0,搜索粒粒面，结果出来的不是粒粒面，搜索有问题,搜索有问题,zh,baby_product,搜索有问题 搜索粒粒面，结果出来的不是粒粒面，搜索有问题,搜索有问题 搜索粒粒面，结果出来的不是粒粒面，搜索有问题 baby_product,搜索粒粒面，结果出来的不是粒粒面，搜索有问题 baby_product,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/train/amazonEN_train_ml.csv', index = False, header=True)

In [ ]:
## cuando procesamos test para tambien conseguir validation 
df_val, df_test = train_test_split(df, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
df_val.shape, df_test.shape

((2500, 45), (2500, 45))

In [ ]:
df_val.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonZH_valid_ml.csv', index = False, header=True)
df_test.to_csv (r'drive/MyDrive/TFG_1/Datasets_procesados/test/amazonZH_test_ml.csv', index = False, header=True)
